In [1]:
import re
import glob
import xlwings as xw
import math
from pathlib import Path
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import plotly.express as px
import statsmodels.api as sm
import pylab as py
import os, cv2, glob, tempfile
import joblib

from scipy import stats
from scipy.stats import pearsonr

import sklearn
from sklearn import datasets
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.datasets import make_blobs

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn import metrics
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
make_scorer = sklearn.metrics.make_scorer
f1 = make_scorer(f1_score, pos_label=1, average="binary")
from sklearn.metrics import classification_report
from sklearn.inspection import permutation_importance
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import RepeatedStratifiedKFold

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import Adam
from keras import utils as np_utils
from keras import models
import keras_tuner as kt
from scipy import signal

import config as config
print(config.__path__)
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras import backend as K

import argparse
# from sklearn.utils import class_weight
print(tf.__version__)
print(keras.__version__)

import shutil
import xlwings as xw

%matplotlib inline
mpl.use("Agg")

['C:\\Users\\user\\anaconda3\\lib\\site-packages\\config']
2.12.0-dev20221107
2.12.0


In [2]:
## Build XGBoost Model
from copy import deepcopy
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score,f1_score,roc_auc_score,confusion_matrix,roc_curve
import xgboost as xgb
import time
import warnings

# 1. Data Preparation

In [3]:
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_dbMSIn10ppm4nonInDI_norm.csv")
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_dbMSIn10ppm4nonInDI_norm.csv")

In [4]:
trainDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,280.2361,241.2173,311.2228,339.1996,353.2005,325.1842,250.1449,514.2846,265.1478,type
0,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.318754,0.131312,-0.119107,-0.067581,-0.070959,0.384385,-0.143321,0.117940,-0.082549,0.226487,-0.001888,0.230784,0.249560,0.293673,0.126161,-0.041332,0.322933,0
1,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.440844,0.345871,-0.038275,-0.067581,-0.041906,0.479853,-0.160301,0.058039,-0.102401,0.292139,-0.001888,0.312511,0.313451,0.399217,0.300473,-0.041332,0.392722,0
2,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.297849,0.145259,-0.083683,-0.067581,-0.068710,0.317078,-0.150702,-0.029642,-0.088098,0.210483,-0.001888,0.229704,0.257451,0.267804,0.208796,-0.041332,0.315267,0
3,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.276313,0.100429,-0.042440,-0.067581,0.086037,0.299038,-0.143072,-0.018275,-0.106710,0.272697,-0.001888,0.221047,0.208237,0.247310,0.187167,-0.041332,0.303835,0
4,HKULiver_Post29wk_HCCAMCLiver_Left_Mice03_Slid...,0.172624,0.175081,-0.131905,-0.067581,-0.053379,0.219193,-0.146098,0.139855,-0.064403,0.166685,-0.001888,0.138936,0.193953,0.151398,0.094787,-0.041332,0.213323,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90955,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.035484,-0.239962,0.004162,-0.067581,-0.134644,0.062940,-0.200323,0.242535,-0.173635,-0.248818,-0.001888,0.042844,0.066221,0.035221,0.038172,0.211987,0.036052,1
90956,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.021818,-0.239962,-0.091238,-0.067581,-0.062125,0.074277,-0.191712,0.300273,-0.195074,0.152613,-0.001888,0.050352,0.070108,0.037013,0.210626,-0.041332,0.018590,1
90957,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.030898,0.291342,-0.093392,-0.067581,-0.088137,0.044543,-0.114220,0.195582,-0.156608,-0.248818,-0.001888,0.046825,0.102873,0.037403,0.143300,-0.041332,-0.052554,1
90958,HKULiver_Post29wk_HCCLiver_Left_Mice12_Slide4F...,0.045515,-0.239962,-0.122675,-0.067581,-0.153338,0.059960,-0.138425,0.149460,-0.163003,-0.248818,-0.001888,0.033919,-0.067732,0.036383,0.189035,-0.041332,0.048042,1


In [5]:
can = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(trainDEFSDf.columns))-1]:
    can.append(list(trainDEFSDf.columns)[i])

dican = []
for i in list(range(0,9)) + list(range(15,17)) + [len(list(diDEFSDf.columns))-1]:
    dican.append(list(diDEFSDf.columns)[i])

In [6]:
can

['pixel_id',
 '311.1684',
 '269.2486',
 '215.0328',
 '295.2278',
 '883.533',
 '309.1704',
 '738.5066',
 '435.2965',
 '250.1449',
 '514.2846',
 'type']

In [7]:
dican

['pixel_id',
 '311.169',
 '269.2486',
 '215.0305',
 '295.2279',
 '883.5375',
 '309.1728',
 '738.5084',
 '435.2966',
 '250.145',
 '514.2844',
 'type']

In [8]:
## input training set ## 90960 x 20 df
trainDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_train_dbMSIn10ppm4nonInDI_norm.csv")
#trainDEFSDf[trainDEFSDf.type .== 1, :]
## calculate weight ## 0: 47449, 1: 43511
trainDEFSDf = trainDEFSDf[can]
Yy_train = trainDEFSDf["type"]  # 0.9585; 1.0453
sampleW = []
for w in Yy_train:
    if w == 0:
        sampleW.append(0.9585)
    elif w == 1:
        sampleW.append(1.0453) 


In [9]:
## input ext val set ## 6075 x 20 df
extDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ext_dbMSIn10ppm4nonInDI_norm.csv")
#extDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 2943, 1: 3132
extDEFSDf = extDEFSDf[can]
Yy_ext = extDEFSDf["type"]  # 1.0321; 0.9698
sampleExtW = []
for w in Yy_ext:
    if w == 0:
        sampleExtW.append(1.0321)
    elif w == 1:
        sampleExtW.append(0.9698) 


In [10]:
## input ingested set ## 97035 x 20 df
ingestedDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_ingested_dbMSIn10ppm4nonInDI_norm.csv")
#ingestedDEFSDf[extDEFSDf.type .== 1, :]
## calculate weight ## 0: 50392, 1: 46643
ingestedDEFSDf = ingestedDEFSDf[can]
Yy_ingested = ingestedDEFSDf["type"]  # 0.9628; 1.0402
sampleIngestedW = []
for w in Yy_ingested:
    if w == 0:
        sampleIngestedW.append(0.9628)
    elif w == 1:
        sampleIngestedW.append(1.0402)


In [11]:
## input FNA set ## 88701 x 20 df
fnaDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_FNA_dbMSIn10ppm4nonInDI_norm.csv")
#fnaDEFSDf[fnaDEFSDf.type .== 1, :]
## calculate weight ##  0: 44540, 1: 44161
fnaDEFSDf = fnaDEFSDf[can]
Yy_FNA = fnaDEFSDf["type"]  # 0.9957; 1.0043
sampleFNAW = []
for w in Yy_FNA:
    if w == 0:
        sampleFNAW.append(0.9957)
    elif w == 1:
        sampleFNAW.append(1.0043)


In [14]:
## input DirectIn set ## 88701 x 20 df
diDEFSDf = pd.read_csv(r"H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\df_nonInDI_dbMSIn10ppm4nonInDI_norm.csv")
#diDEFSDf[diDEFSDf.type .== 1, :]
## calculate weight ## 0: 3027, 1: 3030
diDEFSDf = diDEFSDf[dican]
diDEFSDf = diDEFSDf.rename(columns={"311.169": "311.1684", "215.0305":"215.0328", "295.2279":"295.2278", "883.5375":"883.533", 
                                    "309.1728":"309.1704", "738.5084":"738.5066", "435.2966":"435.2965", "280.2364":"280.2361", "241.217":"241.2173", 
                                    "311.2224":"311.2228", "339.1999":"339.1996", "353.1996":"353.2005", "325.1844":"325.1842", "250.145":"250.1449", 
                                    "514.2844":"514.2846", "265.1479":"265.1478"})
Yy_DI = diDEFSDf["type"]  # 1.0005; 0.9995
sampleDiW = []
for w in Yy_DI:
    if w == 0:
        sampleDiW.append(1.0005)
    elif w == 1:
        sampleDiW.append(0.9995)


In [15]:
diDEFSDf

,pixel_id,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846,type
0,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.005233,0.142288,0.039170,0.003879,0.077996,0.211155,0.061852,0.125873,0.467335,0.036498,0
1,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.089591,0.156412,0.174829,0.001265,0.167022,0.317443,0.127993,0.145532,0.583291,0.099618,0
2,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.032985,0.134206,0.034411,-0.069655,0.121979,0.212954,0.028980,0.076831,0.465178,0.056825,0
3,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.081001,0.235152,0.061915,0.057514,0.129092,0.303459,0.108110,0.193013,0.550683,0.090339,0
4,HKULiver_Post29wk_HCCAMCLiver_DirectIn_MiceNL_...,0.132548,0.236938,0.152597,0.080273,0.239269,0.394214,0.236241,0.204562,0.645485,0.147333,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6052,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.002046,-0.209455,0.152213,-0.039157,-0.042523,-0.056631,0.023261,0.147171,-0.089381,0.163267,1
6053,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.313188,-0.217983,-0.308808,-0.299289,-0.273013,-0.303023,-0.280535,-0.277840,-0.025246,-0.251944,1
6054,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.158181,-0.037213,-0.000751,-0.048526,-0.045113,-0.153125,-0.084896,-0.039408,-0.089381,0.091528,1
6055,HKULiver_Post29wk_HCCLiver_DirectIn_MiceHCC_S1...,-0.130111,-0.100314,0.065751,-0.107947,-0.006814,-0.127231,-0.068984,-0.070642,-0.010182,-0.018040,1


In [16]:
## define functions for performace evaluation ##

# Average score
def avgScore(arrAcc, cv):
    sumAcc = 0
    for acc in arrAcc:
        sumAcc += acc
    return sumAcc / cv


In [17]:
trainDEFSDf[trainDEFSDf.columns[1:-1]]

,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846
0,0.318754,0.131312,-0.119107,-0.067581,-0.070959,0.384385,-0.143321,0.117940,0.126161,-0.041332
1,0.440844,0.345871,-0.038275,-0.067581,-0.041906,0.479853,-0.160301,0.058039,0.300473,-0.041332
2,0.297849,0.145259,-0.083683,-0.067581,-0.068710,0.317078,-0.150702,-0.029642,0.208796,-0.041332
3,0.276313,0.100429,-0.042440,-0.067581,0.086037,0.299038,-0.143072,-0.018275,0.187167,-0.041332
4,0.172624,0.175081,-0.131905,-0.067581,-0.053379,0.219193,-0.146098,0.139855,0.094787,-0.041332
...,...,...,...,...,...,...,...,...,...,...
90955,0.035484,-0.239962,0.004162,-0.067581,-0.134644,0.062940,-0.200323,0.242535,0.038172,0.211987
90956,0.021818,-0.239962,-0.091238,-0.067581,-0.062125,0.074277,-0.191712,0.300273,0.210626,-0.041332
90957,0.030898,0.291342,-0.093392,-0.067581,-0.088137,0.044543,-0.114220,0.195582,0.143300,-0.041332
90958,0.045515,-0.239962,-0.122675,-0.067581,-0.153338,0.059960,-0.138425,0.149460,0.189035,-0.041332


In [18]:
import warnings
warnings.filterwarnings('ignore')

# 2. Unsupervised Learning

## 2.1. Dataset Preparation

In [19]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [20]:
df_ROI_for_ML_Opti = pd.concat([trainDEFSDf, extDEFSDf, fnaDEFSDf, diDEFSDf]).set_index('pixel_id')

In [21]:
df_ROI_for_ML_Opti_ingested = ingestedDEFSDf.set_index('pixel_id')

In [22]:
df_ROI_for_ML_Opti_train = trainDEFSDf.set_index('pixel_id')

In [23]:
df_ROI_for_ML_Opti_ext = extDEFSDf.set_index('pixel_id')

In [24]:
df_ROI_for_ML_Opti_FNA = fnaDEFSDf.set_index('pixel_id')

In [25]:
df_ROI_for_ML_Opti_di = diDEFSDf.set_index('pixel_id')

In [26]:
df_ROI_for_ML_Opti_di.describe()

,311.1684,269.2486,215.0328,295.2278,883.533,309.1704,738.5066,435.2965,250.1449,514.2846,type
count,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6.057000e+03,6057.000000
mean,2.786097e-18,6.620646e-17,-1.465725e-16,-1.438873e-18,-5.146581e-16,9.102472e-17,5.544700e-17,-1.341561e-15,-1.901503e-15,-7.857894e-17,0.500248
std,1.259709e-01,1.439456e-01,1.669524e-01,1.354221e-01,1.815312e-01,1.711848e-01,1.276838e-01,2.073881e-01,1.914128e-01,1.330110e-01,0.500041
min,-3.936976e-01,-4.211962e-01,-3.088082e-01,-4.545066e-01,-2.730134e-01,-3.622890e-01,-2.805354e-01,-2.778402e-01,-8.938063e-02,-3.327803e-01,0.000000
25%,-7.672668e-02,-6.810890e-02,-9.627632e-02,-6.188994e-02,-1.070219e-01,-8.982515e-02,-5.995209e-02,-2.778402e-01,-8.938063e-02,-6.633725e-02,0.000000
50%,9.552356e-03,-6.986512e-03,1.591529e-02,1.774746e-02,3.089123e-02,-3.331810e-02,1.460550e-02,6.108422e-03,-8.938063e-02,-1.853303e-02,1.000000
75%,9.000432e-02,5.958404e-02,1.130540e-01,7.451773e-02,1.304366e-01,3.533862e-02,6.956954e-02,1.665798e-01,-1.203494e-03,8.938771e-02,1.000000
max,6.063024e-01,5.788038e-01,6.911918e-01,5.454934e-01,7.269866e-01,6.377110e-01,7.194646e-01,7.221598e-01,9.106194e-01,6.672197e-01,1.000000


## 2.2. PCA Analysis

In [39]:
## PCA Analysis

def pca_visual(df=df_ROI_for_ML_Opti_di):

    pca = PCA(n_components=3)
    raMSI_ML_mz_df = df.drop("type", axis = 1)
    pca.fit(raMSI_ML_mz_df)
    result = pd.DataFrame(pca.transform(raMSI_ML_mz_df), columns=['PCA%i' % i for i in range(3)], index=df_ROI_for_ML_Opti_di.index)
    
    plt.subplots(dpi = 300)
    plt.scatter(result['PCA1'], result['PCA2'], c = df['type'], s=1)
    plt.xlabel('PC1', size=20)
    plt.ylabel('PC2', size=20)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_10ppm/PCA_di.tif", bbox_inches = 'tight')

    components = pca.fit_transform(raMSI_ML_mz_df)
    total_var = pca.explained_variance_ratio_.sum() * 100
    fig = px.scatter_3d(
        components, x=0, y=1, z=2, color=df['type'],
        title = f'Total Explained Variance: {total_var:.2f}%',
        labels = {'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}
    )
    fig.show()

In [ ]:
pca_visual()

# 3. Supervised Learning

## 3.1. Preparation of training and test dataset

In [41]:
def split_dataset(df=df_ROI_for_ML_Opti_ingested):
    raMSI_ML_NoType = df.drop("type", axis = 1)
    raMSI_ML_Type=df[['type']]
    X_train,X_test,y_train,y_test=train_test_split(raMSI_ML_NoType,raMSI_ML_Type,test_size=0.2,random_state=42)
    return X_train,X_test,y_train,y_test

X_train,X_test,y_train,y_test=split_dataset(df=df_ROI_for_ML_Opti)

In [42]:
X_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[:-1]]

In [43]:
y_ingested = df_ROI_for_ML_Opti_ingested[df_ROI_for_ML_Opti_ingested.columns[-1]]

In [44]:
X_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[:-1]]

In [45]:
y_train = df_ROI_for_ML_Opti_train[df_ROI_for_ML_Opti_train.columns[-1]]

In [46]:
X_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[:-1]]

In [47]:
y_val = df_ROI_for_ML_Opti_ext[df_ROI_for_ML_Opti_ext.columns[-1]]

In [48]:
X_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[:-1]]

In [49]:
y_FNA = df_ROI_for_ML_Opti_FNA[df_ROI_for_ML_Opti_FNA.columns[-1]]

In [50]:
X_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[:-1]]

In [51]:
y_di = df_ROI_for_ML_Opti_di[df_ROI_for_ML_Opti_di.columns[-1]]

## 3.2 Machine Learning Modeling: Model 1+2 (Train 10:10, Val 4:4, Test 3:3)

In [52]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.grid(visible=None)
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

### 3.2.1. Logistic Regression

In [53]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [54]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8395944755652798
Training Set MCC: 0.6786458112529704
Training Set Accuracy: 0.8392480211081794
              precision    recall  f1-score   support

           0       0.85      0.84      0.84     47449
           1       0.83      0.84      0.83     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[39745  7704]
 [ 6918 36593]]
Ext Val Set F1: 0.8306270594579134
Ext Val Set MCC: 0.6395177312090337
Ext Val Set Accuracy: 0.8149794238683128
              precision    recall  f1-score   support

           0       0.90      0.70      0.79      2943
           1       0.77      0.92      0.84      3132

    accuracy                           0.81      6075
   macro avg       0.83      0.81      0.81      6075
weighted avg       0.83      0.81      0.81      6075

[[2055  888]
 [ 236 2896]]
5F-CV: 0.6492343958977618
FNA Set F1: 0.746245976024504
FNA S

### 3.2.2. SVM

In [55]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [56]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9003265850871317
Training Set MCC: 0.7978744229890032
Training Set Accuracy: 0.8980430958663148
              precision    recall  f1-score   support

           0       0.92      0.88      0.90     47449
           1       0.88      0.91      0.90     43511

    accuracy                           0.90     90960
   macro avg       0.90      0.90      0.90     90960
weighted avg       0.90      0.90      0.90     90960

[[41887  5562]
 [ 3712 39799]]
Ext Val Set F1: 0.812745275265812
Ext Val Set MCC: 0.6186819675874873
Ext Val Set Accuracy: 0.8097119341563787
              precision    recall  f1-score   support

           0       0.81      0.79      0.80      2943
           1       0.81      0.83      0.82      3132

    accuracy                           0.81      6075
   macro avg       0.81      0.81      0.81      6075
weighted avg       0.81      0.81      0.81      6075

[[2324  619]
 [ 537 2595]]
5F-CV: 0.9069033263960946
FNA Set F1: 0.642339661931762
FNA Se

### 3.2.3. GBM

In [57]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, yTrain=y_ingested, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [58]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.8813222638278536
Training Set MCC: 0.7619776989149408
Training Set Accuracy: 0.8808597185576077
              precision    recall  f1-score   support

           0       0.89      0.88      0.88     47449
           1       0.87      0.88      0.88     43511

    accuracy                           0.88     90960
   macro avg       0.88      0.88      0.88     90960
weighted avg       0.88      0.88      0.88     90960

[[41667  5782]
 [ 5055 38456]]
Ext Val Set F1: 0.5913130945602354
Ext Val Set MCC: 0.07942430145309565
Ext Val Set Accuracy: 0.5423868312757202
              precision    recall  f1-score   support

           0       0.54      0.41      0.46      2943
           1       0.55      0.67      0.60      3132

    accuracy                           0.54      6075
   macro avg       0.54      0.54      0.53      6075
weighted avg       0.54      0.54      0.53      6075

[[1203 1740]
 [1040 2092]]
5F-CV: 0.6146086959396755
FNA Set F1: 0.6832210749795894
FNA

### 3.2.4. Stacking

In [59]:
class StackingRegressor():
    
    def __init__(self, learners):
        self.level_sizes = []
        self.learners = []
        
        for learning_level in learners:
            self.level_sizes.append(len(learning_level))
            level_learners = []
            
            for learner in learning_level:
                level_learners.append(deepcopy(learner))
            self.learners.append(level_learners)
            
            
    def fit(self, x, y):
        
        meta_data = [x,x,x]
        meta_targets = [y,y,y]
        
        for i in range(len(self.learners)):
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            target_z = np.zeros(len(x))
            
            train_x = meta_data[i]
            train_y = meta_targets[i]
            
            # Define number of folds
            num_folds = 5
            
            # Create the k-fold cross-validation object
            KF = KFold(n_splits=num_folds)
            m = 0

            # Loop over each fold of the cross-validation
            for train_indices, test_indices in KF.split(x):
                
                for j in range(len(self.learners[i])):
                    train_x = pd.DataFrame(train_x)
                    train_y = pd.DataFrame(train_y)

                    learner = self.learners[i][j]
                    learner.fit(train_x.iloc[train_indices], train_y.iloc[train_indices])
                    p = learner.predict(train_x.iloc[test_indices])
                    data_z[j][m: m+len(test_indices) ] = p


                train_y = pd.DataFrame(train_y)
                train_y_array = (np.array(train_y)).reshape(-1,)
                zty_ind = (np.array(train_y.iloc[test_indices])).reshape(-1,)
                target_z[m: m+len(test_indices)] = train_y_array[zty_ind]
                m += len(test_indices)
                

            data_z = data_z.transpose()
            meta_data.append(data_z)
            meta_targets.append(target_z)
            
            
            for learner in self.learners[i]:
                train_x = pd.DataFrame(train_x)
                train_y = pd.DataFrame(train_y)
                learner.fit(train_x, train_y)
                
                
    def predict(self, x):
        
        meta_data = [x,x,x]
        for i in range(len(self.learners)):
            
            level_size = self.level_sizes[i]
            
            data_z = np.zeros((level_size, len(x)))
            
            test_x = meta_data[i]
            
            for j in range(len(self.learners[i])):
                
                learner = self.learners[i][j]
                predictions = learner.predict(test_x)
                data_z[j] = predictions
                
            
            data_z = data_z.transpose()
            meta_data.append(data_z)
            
        return meta_data


In [60]:
## Build stacking Model

def Stacking_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    #meta_learner = LinearDiscriminantAnalysis()  # 0.90,0.80; 0.75,0.51; 0.65,0.27; 0.64,0.26
    #meta_learner = QuadraticDiscriminantAnalysis()  # 0.93,0.86; 0.67,0.39; 0.67,0.15; 0.68,0.20
    
    #meta_learner = LogisticRegression()  # 0.90,0.79; 0.82,0.65; 0.64,0.24; 0.64;0.27
    #meta_learner = LogisticRegression(C=1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.27
    #meta_learner = LogisticRegression(C=0.1, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.1: 0.90,0.79; 0.82,0.65; 0.65,0.25; 0.65;0.28
    #meta_learner = LogisticRegression(C=0.01, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.01: 0.89,0.78; 0.81,0.63; 0.63,0.22; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.0075, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0075: 0.89,0.77; 0.81,0.63; 0.62,0.20; 0.65;0.29
    #meta_learner = LogisticRegression(C=0.005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.005: 0.88,0.76; 0.82,0.64; 0.60,0.17; 0.66;0.31
    #meta_learner = LogisticRegression(C=0.0025, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0025: 0.86,0.72; 0.83,0.66; 0.53,0.02; 0.64;0.31
    #meta_learner = LogisticRegression(C=0.0015, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0015: 0.85,0.70; 0.80,0.60; 0.55,0.09; 0.64;0.30
    #meta_learner = LogisticRegression(C=0.0013, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0013: 0.85,0.70; 0.78,0.56; 0.58,0.16; 0.63;0.29
    #meta_learner = LogisticRegression(C=0.0012, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0012: 0.85,0.69; 0.77,0.52; 0.60,0.22; 0.63;0.27
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0011: 0.85,0.69; 0.77,0.51; 0.64,0.30; 0.62;0.25
    #meta_learner = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.001: 0.84,0.68; 0.76,0.49; 0.68,0.38; 0.61;0.22
    #meta_learner = LogisticRegression(C=0.0005, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)
    ## C=0.0005: 0.83,0.66; 0.74,0.45; 0.65,0.37; 0.60;0.18
    
    #meta_learner = LinearSVC()  # 0.90,0.80; 0.82,0.65; 0.64,0.24; 0.65,0.27
    #meta_learner = KNeighborsClassifier()  # 0.98,0.96; 0.65,0.30; 0.68,0.21; 0.52,-0.05
    #meta_learner = DecisionTreeClassifier()  # 0.94,0.87; 0.54,0.15; 0.66,0.11; 0.46,-0.11
    #meta_learner = RandomForestClassifier()  # 0.98,0.95; 0.54,0.20; 0.66,0.11; 0.49;-0.01
    #meta_learner = GradientBoostingClassifier()  # 0.97,0.94; 0.56,0.27; 0.66,0.13; 0.48,-0.09
    #meta_learner = AdaBoostClassifier()  # 0.96,0.93; 0.69,0.42; 0.62,0.10; 0.54,0.11
    #meta_learner = xgb.XGBClassifier()  # 0.98,0.95; 0.59,0.30; 0.66,0.10; 0.44,-0.12

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [61]:
Stacking_func()

Training Set F1: 0.8414593941362997
Training Set MCC: 0.682027692677651
Training Set Accuracy: 0.8408861037818821
              precision    recall  f1-score   support

           0       0.85      0.84      0.85     47449
           1       0.83      0.84      0.84     43511

    accuracy                           0.84     90960
   macro avg       0.84      0.84      0.84     90960
weighted avg       0.84      0.84      0.84     90960

[[39771  7678]
 [ 6795 36716]]
Ext Val Set F1: 0.849352626132239
Ext Val Set MCC: 0.6828597635811187
Ext Val Set Accuracy: 0.8388477366255144
              precision    recall  f1-score   support

           0       0.90      0.75      0.82      2943
           1       0.80      0.92      0.86      3132

    accuracy                           0.84      6075
   macro avg       0.85      0.84      0.84      6075
weighted avg       0.85      0.84      0.84      6075

[[2202  741]
 [ 238 2894]]
FNA Set F1: 0.7151243046654998
FNA Set MCC: 0.44648309319184415

### 3.2.5. Voting

In [62]:
# get a voting ensemble of models
def get_voting():
    models = list()
    # Define the base models
    models.append(("m1", LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m2", LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})))
    models.append(("m3", GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)))

    # define the voting ensemble
    ensemble = VotingClassifier(estimators=models, voting='hard')
    return ensemble

# get a list of models to evaluate
def get_models():
    models = dict()
    models['m1'] = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m2'] = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    models['m3'] = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    models['hard_voting'] = get_voting()
    return models


# evaluate a give model using cross-validation
def evaluate_model(model, XX, yy):
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)
    scores = cross_val_score(model, XX, yy, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
    return scores


In [63]:
from matplotlib.pyplot import figure
plt.rcParams['figure.figsize'] = [10, 10]
plt.close()
figure(figsize=(10, 10))

<Figure size 720x720 with 0 Axes>

In [65]:
# get the models to evaluate
models = get_models()

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, XX=X_ingested, yy=y_ingested)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
    
# plot model performance for comparison
plt.subplots(dpi = 600)
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('F1 score')
plt.xlabel('Predictive Models')
plt.savefig(r"H:/3_output_raMSIn/3_3_Output_raMSIn_HKU_Ingested4ALL/XGB_ALL/afterModelSelection/dbMSIn_10ppm/voting.jpg", bbox_inches = 'tight')

>m1 0.842 (0.002)
>m2 0.931 (0.011)
>m3 0.647 (0.154)
>hard_voting 0.913 (0.022)


In [66]:
def Voting_func(XTrain=X_ingested, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_ingested, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [67]:
Voting_func()

Training Set F1: 0.9006116619546602
Training Set MCC: 0.7982371165109023
Training Set Accuracy: 0.8981200527704486
              precision    recall  f1-score   support

           0       0.92      0.88      0.90     47449
           1       0.88      0.92      0.90     43511

    accuracy                           0.90     90960
   macro avg       0.90      0.90      0.90     90960
weighted avg       0.90      0.90      0.90     90960

[[41823  5626]
 [ 3641 39870]]
Ext Val Set F1: 0.8188270736425757
Ext Val Set MCC: 0.6116736930714851
Ext Val Set Accuracy: 0.7996707818930041
              precision    recall  f1-score   support

           0       0.89      0.67      0.76      2943
           1       0.75      0.92      0.83      3132

    accuracy                           0.80      6075
   macro avg       0.82      0.80      0.79      6075
weighted avg       0.82      0.80      0.80      6075

[[1966  977]
 [ 240 2892]]
5F-CV: 0.8328046334626574
FNA Set F1: 0.7086498547038445
FNA 

In [68]:
import joblib as jl

In [69]:
models = get_models()
    
ensemble = models["hard_voting"]
ensemble.fit(X_ingested, y_ingested)

VotingClassifier(estimators=[('m1',
                              LogisticRegression(C=0.001,
                                                 class_weight={0: 0.9628,
                                                               1: 1.0402},
                                                 max_iter=5000, penalty='l1',
                                                 random_state=42,
                                                 solver='liblinear')),
                             ('m2',
                              LinearSVC(C=461,
                                        class_weight={0: 0.9628, 1: 1.0402},
                                        random_state=42)),
                             ('m3',
                              GradientBoostingClassifier(learning_rate=4,
                                                         max_depth=7,
                                                         min_samples_leaf=4,
                                                         min_sampl

In [70]:
modelSavePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\model3to1Voting_dbMSIn10ppm.joblib"
jl.dump(ensemble, modelSavePath, compress = 5)

['H:\\3_output_raMSIn\\3_3_Output_raMSIn_HKU_Ingested4ALL\\XGB_ALL\\afterModelSelection\\dbMSIn_10ppm\\model3to1Voting_dbMSIn10ppm.joblib']

## 3.3 Machine Learning Modeling: Model 1 (Train 6:6, Val 4:4, Test 3:3)

### 3.3.1. Logistic Regression

In [71]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_lr.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [72]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.8626354283195081
Training Set MCC: 0.7208380477933968
Training Set Accuracy: 0.8594217238346525
              precision    recall  f1-score   support

           0       0.88      0.84      0.86     47449
           1       0.84      0.88      0.86     43511

    accuracy                           0.86     90960
   macro avg       0.86      0.86      0.86     90960
weighted avg       0.86      0.86      0.86     90960

[[39972  7477]
 [ 5310 38201]]
Ext Val Set F1: 0.7108616941245735
Ext Val Set MCC: 0.3815174649728782
Ext Val Set Accuracy: 0.6908641975308641
              precision    recall  f1-score   support

           0       0.71      0.61      0.66      2943
           1       0.68      0.77      0.72      3132

    accuracy                           0.69      6075
   macro avg       0.69      0.69      0.69      6075
weighted avg       0.69      0.69      0.69      6075

[[1798 1145]
 [ 733 2399]]
5F-CV: 0.7392342895086512
FNA Set F1: 0.5123468885333252
FNA 

### 3.3.2. SVM

In [73]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_svm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [74]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.9414395835467654
Training Set MCC: 0.8845813571011703
Training Set Accuracy: 0.9426890941073
              precision    recall  f1-score   support

           0       0.94      0.95      0.95     47449
           1       0.95      0.93      0.94     43511

    accuracy                           0.94     90960
   macro avg       0.94      0.94      0.94     90960
weighted avg       0.94      0.94      0.94     90960

[[45287  2162]
 [ 3051 40460]]
Ext Val Set F1: 0.5613349019737868
Ext Val Set MCC: 0.04610370792258176
Ext Val Set Accuracy: 0.5254320987654321
              precision    recall  f1-score   support

           0       0.51      0.43      0.47      2943
           1       0.53      0.61      0.57      3132

    accuracy                           0.53      6075
   macro avg       0.52      0.52      0.52      6075
weighted avg       0.52      0.53      0.52      6075

[[1279 1664]
 [1219 1913]]
5F-CV: 0.9078487242948088
FNA Set F1: 0.5917288299699253
FNA Se

### 3.3.3. GBM

In [75]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, yTrain=y_train, yVal=y_val, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(X_train)
    print("Training Set F1:", f1_score(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_gbm.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [76]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.7246297149354756
Training Set MCC: 0.4872951017370456
Training Set Accuracy: 0.744448109058927
              precision    recall  f1-score   support

           0       0.73      0.80      0.77     47449
           1       0.76      0.68      0.72     43511

    accuracy                           0.74     90960
   macro avg       0.75      0.74      0.74     90960
weighted avg       0.75      0.74      0.74     90960

[[38147  9302]
 [13943 29568]]
Ext Val Set F1: 0.45946935163577807
Ext Val Set MCC: 0.032385799963275036
Ext Val Set Accuracy: 0.5125925925925926
              precision    recall  f1-score   support

           0       0.50      0.62      0.55      2943
           1       0.54      0.41      0.47      3132

    accuracy                           0.51      6075
   macro avg       0.52      0.52      0.51      6075
weighted avg       0.52      0.51      0.51      6075

[[1825 1118]
 [1843 1289]]
5F-CV: 0.6375651252108288
FNA Set F1: 0.6799013086219665
FN

### 3.3.4. Stacking

In [77]:
## Build stacking Model

def Stacking_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(X_train)[-1]
    print("Training Set F1:", f1_score(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, model_sc.predict(X_train)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XVal)[-1]
    print("Ext Val Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [78]:
Stacking_func()

Training Set F1: 0.869811156070839
Training Set MCC: 0.7359051184621347
Training Set Accuracy: 0.8670954265611258
              precision    recall  f1-score   support

           0       0.89      0.85      0.87     47449
           1       0.85      0.88      0.86     43511

    accuracy                           0.87     90960
   macro avg       0.87      0.87      0.87     90960
weighted avg       0.87      0.87      0.87     90960

[[40433  7016]
 [ 5073 38438]]
Ext Val Set F1: 0.7098081285350366
Ext Val Set MCC: 0.4043350153280379
Ext Val Set Accuracy: 0.702716049382716
              precision    recall  f1-score   support

           0       0.70      0.67      0.69      2943
           1       0.70      0.73      0.72      3132

    accuracy                           0.70      6075
   macro avg       0.70      0.70      0.70      6075
weighted avg       0.70      0.70      0.70      6075

[[1985  958]
 [ 848 2284]]
FNA Set F1: 0.49758030590701874
FNA Set MCC: -0.041072609270539

### 3.3.5. Voting

In [79]:
def Voting_func(XTrain=X_train, XVal=X_val, XTest=X_FNA, XTest2=X_di, yTrain=y_train, yVal=y_val, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(X_train)
    print("Training Set F1:", f1_score(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(y_train, ensemble.predict(X_train), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(y_train, X_train_predictions))
    print(classification_report(y_train, X_train_predictions))
    print(confusion_matrix(y_train, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(y_train, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XVal)
    print("Ext Val Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yVal, predictions_val))
    print(classification_report(yVal, predictions_val))
    print(confusion_matrix(yVal, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yVal, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [80]:
Voting_func()

Training Set F1: 0.9349270040977576
Training Set MCC: 0.8736506743695349
Training Set Accuracy: 0.9373570800351803
              precision    recall  f1-score   support

           0       0.92      0.96      0.94     47449
           1       0.95      0.91      0.93     43511

    accuracy                           0.94     90960
   macro avg       0.94      0.94      0.94     90960
weighted avg       0.94      0.94      0.94     90960

[[45495  1954]
 [ 3744 39767]]
Ext Val Set F1: 0.6596463145197563
Ext Val Set MCC: 0.28922876145103016
Ext Val Set Accuracy: 0.6454320987654321
              precision    recall  f1-score   support

           0       0.64      0.60      0.62      2943
           1       0.65      0.69      0.67      3132

    accuracy                           0.65      6075
   macro avg       0.65      0.64      0.64      6075
weighted avg       0.65      0.65      0.64      6075

[[1760 1183]
 [ 971 2161]]
5F-CV: 0.8525316226330972
FNA Set F1: 0.5723581790867517
FNA

## 3.4 Machine Learning Modeling: Model 2 (Train 4:4, Val 6:6, Test 3:3)

### 3.4.1. Logistic Regression

In [81]:
## Build Logistic Regression Model
from sklearn.linear_model import LogisticRegression

def LogisticRegression_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_lr = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})  # max_iter=100
    model_lr.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_lr.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_lr.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_lr.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_lr.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_lr, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_lr.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_lr.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_lr.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_lr.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_lr.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [82]:
coeff_LogReg, TOPfeatures_LogReg = LogisticRegression_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
5F-CV: 0.0
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              

### 3.4.2. SVM

In [83]:
## Build LinearSVC Model
from sklearn.svm import LinearSVC

def LinearSVC_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_svm = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    model_svm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_svm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_svm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_svm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_svm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_svm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_svm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_svm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_svm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_svm.coef_[0], index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()
    
    #feature_importances = model_svm.coef_[0]
    #feature_df=pd.DataFrame({'m/z':XTrain.columns,'coef':feature_importances})

    #fig = px.bar(feature_df, x='coef', y='m/z')
    #fig.show()
    
    return x, y

In [84]:
coeff_SVM, TOPfeatures_SVM = LinearSVC_func()

Training Set F1: 0.6587199150306594
Training Set MCC: 0.19619229957442816
Training Set Accuracy: 0.5809696569920845
              precision    recall  f1-score   support

           0       0.67      0.39      0.49     47449
           1       0.54      0.79      0.64     43511

    accuracy                           0.58     90960
   macro avg       0.61      0.59      0.57     90960
weighted avg       0.61      0.58      0.56     90960

[[18390 29059]
 [ 9056 34455]]
Ext Val Set F1: 0.9585720391445273
Ext Val Set MCC: 0.9182466972197648
Ext Val Set Accuracy: 0.9586831275720165
              precision    recall  f1-score   support

           0       0.95      0.97      0.96      2943
           1       0.97      0.95      0.96      3132

    accuracy                           0.96      6075
   macro avg       0.96      0.96      0.96      6075
weighted avg       0.96      0.96      0.96      6075

[[2854   89]
 [ 162 2970]]
5F-CV: 0.900472611965862
FNA Set F1: 0.4936553434625675
FNA 

### 3.4.3. GBM

In [85]:
## Build GBM Model
from sklearn.ensemble import GradientBoostingClassifier

def GBM_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, yTrain=y_val, yVal=y_train, yTest=y_FNA):
    model_gbm = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    model_gbm.fit(XTrain,yTrain)
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_gbm.predict(XVal)
    print("Training Set F1:", f1_score(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_gbm.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_gbm.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_gbm.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    f1_5_train = cross_val_score(model_gbm, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_gbm.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_gbm.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_gbm.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Visualize the importantance of the features
    feature_imp = pd.Series(model_gbm.feature_importances_, index=XTrain.columns).sort_values(ascending=False)
    x=feature_imp
    y=feature_imp.index
    
    plt.subplots(dpi = 300)
    #plt.rcParams["figure.figsize"] = [4, 6]
    #plt.rcParams["font.family"] = "Arial"
    #plt.rcParams.update({'font.size': 6})
    plt.xlabel('Feature Importance Score', fontproperties='Arial', fontsize=17)
    plt.ylabel('Features', fontproperties='Arial', fontsize=17)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.title("Important Molecular Features", fontproperties='Arial', fontsize=20)
    sns.barplot(x=feature_imp, y=feature_imp.index)
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/XGB_TopFeatures.tif", bbox_inches = 'tight')
    #plt.show()

    # Visualize the boxplot of the important features between the two groups
    #fig = px.box(data_frame = df_ROI_for_ML_Opti, x = feature_imp[:10].index,color= 'type')
    #fig.show()
    
    return x, y

In [86]:
coeff_GBM, TOPfeatures_GBM = GBM_func()

Training Set F1: 0.6272904974098836
Training Set MCC: 0.08638419127473955
Training Set Accuracy: 0.5275175901495163
              precision    recall  f1-score   support

           0       0.59      0.30      0.40     47449
           1       0.50      0.78      0.61     43511

    accuracy                           0.53     90960
   macro avg       0.55      0.54      0.50     90960
weighted avg       0.55      0.53      0.50     90960

[[14144 33305]
 [ 9672 33839]]
Ext Val Set F1: 0.7064311230232555
Ext Val Set MCC: 0.324960968410552
Ext Val Set Accuracy: 0.6559670781893004
              precision    recall  f1-score   support

           0       0.73      0.46      0.56      2943
           1       0.62      0.84      0.72      3132

    accuracy                           0.66      6075
   macro avg       0.68      0.65      0.64      6075
weighted avg       0.68      0.66      0.64      6075

[[1347 1596]
 [ 494 2638]]
5F-CV: 0.6801213220071441
FNA Set F1: 0.5005783441740776
FNA 

### 3.4.4. Stacking

In [87]:
## Build stacking Model

def Stacking_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    base_learners = []

    # Define models to use in stacking ensemble
    m1 = LogisticRegression(C=0.001, penalty='l1', solver="liblinear", max_iter=5000, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m1)

    m2 = LinearSVC(penalty='l2', loss="squared_hinge", C=461, random_state=42, class_weight={0:0.9628, 1:1.0402})
    base_learners.append(m2)

    m3 = GradientBoostingClassifier(learning_rate=4, n_estimators=50, max_depth=7, min_samples_leaf=4, min_samples_split=30, n_iter_no_change=5, random_state=42)
    base_learners.append(m3)

    # Define the meta-classifier to use for stacking
    meta_learner = LogisticRegression(C=0.0011, penalty='l1', solver="liblinear", max_iter=5000, random_state=42)

    model_sc = StackingRegressor([[m1, m2, m3], [meta_learner]])
    model_sc.fit(XTrain, yTrain)
    
    #=======================================
    
    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = model_sc.predict(XVal)[-1]
    print("Training Set F1:", f1_score(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, model_sc.predict(XVal)[-1], sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = model_sc.predict(XTrain)[-1]
    print("Ext Val Set F1:", f1_score(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, model_sc.predict(XTrain)[-1], sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()
    
    #f1_5_train = cross_val_score(model_sc, XTrain, yTrain, cv = 5, scoring="f1")
    #print("5F-CV:", avgScore(f1_5_train, 5))
    
    # Test set
    predictions_FNA = model_sc.predict(XTest)[-1]
    print("FNA Set F1:", f1_score(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, model_sc.predict(XTest)[-1], sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, model_sc.predict(XTest)[-1]))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    
    # Test2 set
    predictions_di = model_sc.predict(XTest2)[-1]
    print("DirectIn Set F1:", f1_score(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, model_sc.predict(XTest2)[-1], sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, model_sc.predict(XTest2)[-1]))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()
    

In [88]:
Stacking_func()

Training Set F1: 0.0
Training Set MCC: 0.0
Training Set Accuracy: 0.5216468777484609
              precision    recall  f1-score   support

           0       0.52      1.00      0.69     47449
           1       0.00      0.00      0.00     43511

    accuracy                           0.52     90960
   macro avg       0.26      0.50      0.34     90960
weighted avg       0.27      0.52      0.36     90960

[[47449     0]
 [43511     0]]
Ext Val Set F1: 0.0
Ext Val Set MCC: 0.0
Ext Val Set Accuracy: 0.48444444444444446
              precision    recall  f1-score   support

           0       0.48      1.00      0.65      2943
           1       0.00      0.00      0.00      3132

    accuracy                           0.48      6075
   macro avg       0.24      0.50      0.33      6075
weighted avg       0.23      0.48      0.32      6075

[[2943    0]
 [3132    0]]
FNA Set F1: 0.0
FNA Set MCC: 0.0
FNA Set Recall: 0.0
FNA Test Set Accuracy: 0.5021363907960451
              precision  

### 3.4.5. Voting

In [89]:
def Voting_func(XTrain=X_val, XVal=X_train, XTest=X_FNA, XTest2=X_di, yTrain=y_val, yVal=y_train, yTest=y_FNA, yTest2=y_di):

    models = get_models()
    
    ensemble = models["hard_voting"]
    ensemble.fit(XTrain, yTrain)
    #=======================================

    # Training Set
    #plt.rcParams["figure.figsize"] = (3,3)
    X_train_predictions = ensemble.predict(XVal)
    print("Training Set F1:", f1_score(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set MCC:", matthews_corrcoef(yVal, ensemble.predict(XVal), sample_weight=sampleW))
    print("Training Set Accuracy:", metrics.accuracy_score(yVal, X_train_predictions))
    print(classification_report(yVal, X_train_predictions))
    print(confusion_matrix(yVal, X_train_predictions))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr, tpr, _ = metrics.roc_curve(yVal, X_train_predictions)
    plt.plot(fpr,tpr)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_train.tif", bbox_inches = 'tight')
    #plt.show()

    # Val set
    predictions_val  = ensemble.predict(XTrain)
    print("Ext Val Set F1:", f1_score(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set MCC:", matthews_corrcoef(yTrain, ensemble.predict(XTrain), sample_weight=sampleExtW))
    print("Ext Val Set Accuracy:", metrics.accuracy_score(yTrain, predictions_val))
    print(classification_report(yTrain, predictions_val))
    print(confusion_matrix(yTrain, predictions_val))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_val, tpr_val, _ = metrics.roc_curve(yTrain, predictions_val)
    plt.plot(fpr_val,tpr_val)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"H:/3_2_Output_raMSI_DataIngestion_DESIQE_12F/XGB_FNA/LogReg_ROC_test.tif", bbox_inches = 'tight')
    #plt.show()

    f1_5_train = cross_val_score(ensemble, XTrain, yTrain, cv = 5, scoring="f1")
    print("5F-CV:", avgScore(f1_5_train, 5))

    # Test set
    predictions_FNA = ensemble.predict(XTest)
    print("FNA Set F1:", f1_score(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set MCC:", matthews_corrcoef(yTest, ensemble.predict(XTest), sample_weight=sampleFNAW))
    print("FNA Set Recall:", recall_score(yTest, ensemble.predict(XTest)))
    print("FNA Test Set Accuracy:", metrics.accuracy_score(yTest, predictions_FNA))
    print(classification_report(yTest, predictions_FNA))
    print(confusion_matrix(yTest, predictions_FNA))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_FNA, tpr_FNA, _ = metrics.roc_curve(yTest, predictions_FNA)
    plt.plot(fpr_FNA,tpr_FNA)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

    # Test2 set
    predictions_di = ensemble.predict(XTest2)
    print("DirectIn Set F1:", f1_score(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set MCC:", matthews_corrcoef(yTest2, ensemble.predict(XTest2), sample_weight=sampleDiW))
    print("DirectIn Set Recall:", recall_score(yTest2, ensemble.predict(XTest2)))
    print("DirectIn Test Set Accuracy:", metrics.accuracy_score(yTest2, predictions_di))
    print(classification_report(yTest2, predictions_di))
    print(confusion_matrix(yTest2, predictions_di))

    #create ROC curve
    plt.subplots(dpi = 300)
    fpr_di, tpr_di, _ = metrics.roc_curve(yTest2, predictions_di)
    plt.plot(fpr_di,tpr_di)
    plt.ylabel('True Positive Rate', fontsize=18)
    plt.xlabel('False Positive Rate', fontsize=18)
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    #plt.rcParams['figure.figsize'] = [2, 2]
    #plt.savefig(r"G:/raMSIn/XGB_Importance2/XGB2_ROC_FNAtest2.tif", bbox_inches = 'tight')
    #plt.show()

In [90]:
Voting_func()

Training Set F1: 0.5669661153635158
Training Set MCC: 0.05868167455669913
Training Set Accuracy: 0.5250769569041337
              precision    recall  f1-score   support

           0       0.56      0.44      0.49     47449
           1       0.50      0.62      0.55     43511

    accuracy                           0.53     90960
   macro avg       0.53      0.53      0.52     90960
weighted avg       0.53      0.53      0.52     90960

[[20923 26526]
 [16673 26838]]
Ext Val Set F1: 0.8896307122341802
Ext Val Set MCC: 0.8093567805073314
Ext Val Set Accuracy: 0.8962962962962963
              precision    recall  f1-score   support

           0       0.83      0.98      0.90      2943
           1       0.98      0.81      0.89      3132

    accuracy                           0.90      6075
   macro avg       0.91      0.90      0.90      6075
weighted avg       0.91      0.90      0.90      6075

[[2893   50]
 [ 580 2552]]
5F-CV: 0.6799125413772102
FNA Set F1: 0.3555746865210758
FNA

# 4. Model Evaluation

## 4.1 Prediction

In [91]:
## deploy models for training set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\model3to1Voting_dbMSIn10ppm.joblib")
#
## deploy model ##
predicted0n1_train = model_voting.predict(X_train)
#predicted0n1_trainProba = model_voting.predict_proba(X_train)         
trainDEFSDf["predicted0n1"] = predicted0n1_train
T_ = []
F_ = []
for i in range(len(trainDEFSDf)):
    if trainDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif trainDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
trainDEFSDf["T"] = T_
trainDEFSDf["F"] = F_
#
## save ##, ouputing trainSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\df_train_dbMSIn10ppm_norm_0n1.csv"
trainDEFSDf.to_csv(savePath)

In [92]:
## deploy models for Ext Val set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\model3to1Voting_dbMSIn10ppm.joblib")
#
## deploy model ##
predicted0n1_ext = model_voting.predict(X_val)
extDEFSDf["predicted0n1"] = predicted0n1_ext
T_ = []
F_ = []
for i in range(len(extDEFSDf)):
    if extDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif extDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
extDEFSDf["T"] = T_
extDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\df_ext_dbMSIn10ppm_norm_0n1.csv"
extDEFSDf.to_csv(savePath)

In [93]:
## deploy models for FNA set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\model3to1Voting_dbMSIn10ppm.joblib")
#
## deploy model ##
predicted0n1_FNA = model_voting.predict(X_FNA)
fnaDEFSDf["predicted0n1"] = predicted0n1_FNA
T_ = []
F_ = []
for i in range(len(fnaDEFSDf)):
    if fnaDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif fnaDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
fnaDEFSDf["T"] = T_
fnaDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\df_FNA_dbMSIn10ppm_norm_0n1.csv"
fnaDEFSDf.to_csv(savePath)

In [94]:
## deploy models for DirectInfusion set ##
model_voting = jl.load(r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\model3to1Voting_dbMSIn10ppm.joblib")
#
## deploy model ##
predicted0n1_Di = model_voting.predict(X_di)
diDEFSDf["predicted0n1"] = predicted0n1_Di
T_ = []
F_ = []
for i in range(len(diDEFSDf)):
    if diDEFSDf.iloc[i]["predicted0n1"] == 1:
        T_.append(1)
        F_.append(0)
    elif diDEFSDf.iloc[i]["predicted0n1"] == 0:
        F_.append(1)
        T_.append(0)
diDEFSDf["T"] = T_
diDEFSDf["F"] = F_
#
## save ##, ouputing extSet df
savePath = r"H:\3_output_raMSIn\3_3_Output_raMSIn_HKU_Ingested4ALL\XGB_ALL\afterModelSelection\dbMSIn_10ppm\df_Di_dbMSIn10ppm_norm_0n1.csv"
diDEFSDf.to_csv(savePath)